In [1]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("gym_members_exercise_tracking.csv")
df.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39


In [16]:
def Relación_entre_BMI_Rendimiento(df: pd.DataFrame) -> dict:
		relacion_rendimiento_calorias_quemadas = df["BMI"].corr(df["Calories_Burned"])
		relacion_rendimiento_avgBPM=df["BMI"].corr(df["Avg_BPM"])
		relacion_rendimiento_duracion=df["BMI"].corr(df["Session_Duration (hours)"])
		df["BMI_Category"] = pd.cut(df["BMI"], bins=[0, 18.5, 24.9, 29.9, float('inf')], labels=['Bajo 0-18', 'Normal 18-24', 'Sobrepeso 24-29', 'Obesidad 29+'])


		grouped_stats = df.groupby(["BMI_Category"]).agg({
				"Calories_Burned": ["mean"],
				"Avg_BPM": ["mean"],
				"Session_Duration (hours)": ["mean"]
		}).reset_index()

		grouped_stats.columns = ['_'.join(col).strip() for col in grouped_stats.columns.values]

		grouped_stats= {
			"result_dataframe":  grouped_stats.to_dict(orient="records"),
			"dataframe": [
				{
					"labels": ["Índice de masa coorporal","Calorias quemadas"],
					"correlation": float(relacion_rendimiento_calorias_quemadas * 100),
					"x": grouped_stats["BMI_Category_"].to_list(),
					"y": grouped_stats["Calories_Burned_mean"].to_list(),
				},
				{
					"labels": ["Niveles de experiencia","Frecuencia cardiaca prom"],
					"correlation": float(relacion_rendimiento_avgBPM * 100),
					"x": grouped_stats["BMI_Category_"].to_list(),
					"y": grouped_stats["Avg_BPM_mean"].to_list(),
				},
				{
					"labels": ["Niveles de experiencia","Duración de sesiones"],
					"correlation": float(relacion_rendimiento_duracion* 100),
					"x": grouped_stats["BMI_Category_"].to_list(),
					"y": grouped_stats["Session_Duration (hours)_mean"].to_list(),
				}
			]
		}
		return grouped_stats




In [17]:
result=Relación_entre_BMI_Rendimiento(df)
result_df = pd.DataFrame(result["result_dataframe"])
corr_calories_burned = result["dataframe"][0]["correlation"]
corr_avg_bpm = result["dataframe"][1]["correlation"]
corr_session_duration = result["dataframe"][2]["correlation"]

# graficar
fig = px.line(result_df,
               title=f"Índice de masa corporal y Calorías Quemadas {corr_calories_burned:.2f} %",
               x="BMI_Category_",
               y="Calories_Burned_mean",
               labels={"BMI_Category_":"Índice de masa coorporal", "Calories_Burned_mean":"Calorías Quemadas"},
               )

fig.show()

fig = px.line(result_df,
               title=f"Índice de masa corporal y BPM Promedio {corr_avg_bpm:.2f} %",
               x="BMI_Category_",
               y="Avg_BPM_mean",
               labels={"BMI_Category_":"Índice de masa corporal", "Avg_BPM_mean":"BPM Promedio"},
               )
fig.show()
fig = px.line(result_df,
               title=f"Índice de masa corporal y Duración de la Sesión {corr_session_duration:.2f} %",
               x="BMI_Category_",
               y="Session_Duration (hours)_mean",
               labels={"BMI_Category_":"Índice de masa corporal", "Session_Duration (hours)_mean":"Duración de la Sesión"},
               )
fig.show()

<ipython-input-16-be34af0584d9>:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

